# 8 Schools

ここでは、Gelmanらの「Bayesian Data Analysis: Sec. 5.5, 2003の8 schoolの例を使います。この例では、8 schoolのSATの成績に対するコーチングの効果を研究しています。

データは次のように与えられます。

In [2]:
import numpy as np

J = 8
y = np.array([28.0, 8.0, -3.0, 7.0, -1.0, 1.0, 18.0, 12.0])
sigma = np.array([15.0, 10.0, 16.0, 11.0, 9.0, 11.0, 10.0, 18.0])

ここで，yは治療効果，sigmaは標準誤差を表しています。この研究では、各学校のグループレベルのパラメータθは、未知の平均muと標準偏差tauを持つ正規分布からサンプリングされると仮定して、階層モデルを構築しました。

これにより、すべての観測データからプールすることで、母集団レベルのパラメータmuとtauを推定することができる一方で、グループレベルのθパラメータを用いて学校間の個人差を考慮することができます。

In [3]:
import numpyro
import numpyro.distributions as dist

def eight_schools(J, sigma, y=None):
    mu = numpyro.sample('mu', dist.Normal(0, 5))
    tau = numpyro.sample('tau', dist.HalfCauchy(5))

    with numpyro.plate('J', J):
        theta = numpyro.sample('theta', dist.Normal(mu, tau))
        numpyro.sample('obs', dist.Normal(theta, sigma), obs=y)

NUTS（No-U-Turn Sampler）を用いてMCMCを実行し、モデルの未知のパラメータの値を推論してみましょう。MCMC.run()でのextra_fields引数の使い方に注意してください。デフォルトでは，MCMCによる推論を実行する際には，事後分布からサンプルのみを収集します．しかし、ポテンシャルエネルギーやサンプルの受入確率などの追加フィールドも、extra_fields引数を使用することで簡単に実現できます。収集可能なフィールドは以下です（[参考](https://num.pyro.ai/en/latest/mcmc.html#numpyro.infer.mcmc.HMCState)）。

- i : イテレーション。これは，ウォームアップ後に0にリセットされます．
- z : 潜在的な部位における値（事後評価からの制約を受けないサンプル）を表す Python のコレクション．
- z_grad : 潜在的なサンプルサイトに対する位置エネルギーの勾配。
- potential_energy : 与えられたzの値で計算された位置エネルギー．
- energy : 現在の状態の位置エネルギーと運動エネルギーの合計。
- r : 現在の運動量を表す変数．これがNoneの場合，各サンプリングステップの最初に，新しい運動量変数が作成されます．
- trajectory_length : 各サンプリングステップでHMCダイナミクスを実行するための時間です。このフィールドはNUTSでは使用されません。
- num_steps : ハミルトン・トラジェクトリーのステップ数（診断用）。NUTSサンプラーでは，tree_depth = np.log2(num_steps).astype(int) + 1 で，軌道の木の深さを計算することができます．
- accept_prob : 提案の受け入れ確率。提案が却下された場合、zは提案に対応しないことに注意してください。
- mean_accept_prob : ウォームアップ適応中やサンプリング（診断用）中の現在のイテレーションまでの平均受け入れ確率。
- diverging : 現在の軌道が発散しているかどうかを示すブール値。
- adapt_state : ウォームアップ中の適応情報を含むHMCAdaptStateの名前付きタプルです。
- step_size : 次の反復で積分器が使用するステップサイズ。
- inverse_mass_matrix : 次の反復処理で使用される逆質量行列。
- mass_matrix_sqrt : 次の繰り返しに利用される，質量行列の平方根．密な質量の場合は，質量行列のコレスキー分解になります．
- rng_key : 反復処理に利用される乱数生成器のシード．

この例では、各サンプルの potential_energy を追加で収集します。

In [41]:
from jax import random
from numpyro.infer import MCMC, NUTS


kernel = NUTS(eight_schools)
mcmc = MCMC(kernel, num_warmup=500, num_samples=1000)

rng_key = random.PRNGKey(0)

mcmc.run(rng_key, J, sigma, y=y, extra_fields=('potential_energy',))

sample: 100%|██████████| 1500/1500 [00:03<00:00, 474.14it/s, 7 steps of size 2.30e-01. acc. prob=0.78] 


ここでいうpotential energyはハミルトニアンの運動量と関係しない部分のことです。これはに負の対数同時確率に対応します。

$$
potential\ energy = -\log f(\theta\vert x)
$$

In [42]:
mcmc.print_summary() 
pe = mcmc.get_extra_fields()['potential_energy']
print('Expected log joint density: {:.2f}'.format(np.mean(-pe)))


                mean       std    median      5.0%     95.0%     n_eff     r_hat
        mu      4.45      3.00      4.48     -0.34      9.35    276.84      1.00
       tau      4.24      3.05      3.34      0.90      8.36    120.61      1.01
  theta[0]      6.44      5.64      5.88     -3.01     14.35    278.14      1.00
  theta[1]      5.14      4.78      5.21     -2.21     12.71    486.72      1.00
  theta[2]      4.00      5.21      4.44     -4.31     11.71    395.57      1.00
  theta[3]      4.78      4.92      4.83     -3.80     11.74    566.18      1.00
  theta[4]      3.62      4.53      3.97     -3.37     11.01    413.17      1.00
  theta[5]      3.81      4.98      4.21     -3.29     12.24    346.15      1.00
  theta[6]      6.52      4.83      6.16     -1.35     13.17    341.63      1.00
  theta[7]      5.12      5.00      4.99     -3.03     12.24    374.90      1.01

Number of divergences: 15
Expected log joint density: -55.46


In [46]:
mcmc.get_extra_fields()["diverging"].sum()

DeviceArray(15, dtype=int32)

- r_hat : 収束しているかの判断に利用される。1.0を超えていると完全に収束しているとは言えないとか。1.1以下だったらいいとか色々聞く。
- n_eff : 有効サンプルサイズ。サンプリングの中で自己相関のないと言えるサンプルの実質的な数を表している。

ここでは特にタウの有効サンプルサイズが小さくてモデルに問題がありそうです。幸いなことに、これは一般的な現象で最パラメータにより修正することができます。NumPyroでは、TransformedDistributionインスタンスと再パラメタライズ効果ハンドラを使用することで、これを簡単に行うことができます。

同じモデルを書き直してみましょう。ただし、θをNormal(mu, tau)からサンプリングする代わりに、AffineTransformを用いて変換された基本Normal(0, 1)分布からサンプリングします。同じものからサンプリングしているはずですが、満遍なくサンプリングできる空間からサンプリングすることで有効なサンプルを獲得することができます。

In [49]:
from numpyro.infer.reparam import TransformReparam

def eight_schools_noncentered(J, sigma, y=None):
    mu = numpyro.sample('mu', dist.Normal(0, 5))
    tau = numpyro.sample('tau', dist.HalfCauchy(5))

    with numpyro.plate('J', J):
        with numpyro.handlers.reparam(config={'theta': TransformReparam()}):
            theta = numpyro.sample(
                'theta',
                dist.TransformedDistribution(dist.Normal(0., 1.),
                                             dist.transforms.AffineTransform(mu, tau)))
        numpyro.sample('obs', dist.Normal(theta, sigma), obs=y)

kernel = NUTS(eight_schools_noncentered)
mcmc = MCMC(kernel, num_warmup=500, num_samples=1000)
rng_key = random.PRNGKey(0)
mcmc.run(rng_key, J, sigma, y=y, extra_fields=('potential_energy',))

mcmc.print_summary(exclude_deterministic=False)
pe = mcmc.get_extra_fields()['potential_energy']
print('Expected log joint density: {:.2f}'.format(np.mean(-pe)))  

sample: 100%|██████████| 1500/1500 [00:03<00:00, 461.84it/s, 7 steps of size 4.61e-01. acc. prob=0.90]



                   mean       std    median      5.0%     95.0%     n_eff     r_hat
           mu      4.23      3.46      4.37     -0.92      9.92    538.30      1.00
          tau      3.73      3.45      2.76      0.00      8.31    495.82      1.00
     theta[0]      6.29      5.87      5.60     -2.57     14.41    921.51      1.00
     theta[1]      4.94      4.67      4.78     -2.07     13.00   1378.75      1.00
     theta[2]      3.59      5.32      3.95     -5.35     11.41    956.04      1.00
     theta[3]      4.80      4.74      4.83     -2.27     12.20   1248.78      1.00
     theta[4]      3.56      4.54      4.06     -4.21     10.05   1114.89      1.00
     theta[5]      3.90      5.07      4.22     -3.76     11.56    751.93      1.00
     theta[6]      6.37      4.95      5.79     -2.11     13.10   1017.87      1.00
     theta[7]      4.67      5.27      4.55     -3.10     12.45    655.86      1.00
theta_base[0]      0.31      0.96      0.33     -1.28      1.91    979.49  

ちなみにNormal, Cauchy, StudentTについては上記のような最パラメータ化を容易に実現するためのものが存在しています。

```python
with numpyro.handlers.reparam(config={'theta': LocScaleReparam(centered=0)}):

    theta = numpyro.sample('theta', dist.Normal(mu, tau))
```

In [52]:
def eight_schools_noncentered2(J, sigma, y=None):
    mu = numpyro.sample('mu', dist.Normal(0, 5))
    tau = numpyro.sample('tau', dist.HalfCauchy(5))

    with numpyro.plate('J', J):
        with numpyro.handlers.reparam(config={'theta': numpyro.infer.reparam.LocScaleReparam(centered=0)}):
            theta = numpyro.sample('theta', dist.Normal(mu, tau))
        numpyro.sample('obs', dist.Normal(theta, sigma), obs=y)

kernel = NUTS(eight_schools_noncentered2)
mcmc = MCMC(kernel, num_warmup=500, num_samples=1000)
rng_key = random.PRNGKey(1)
mcmc.run(rng_key, J, sigma, y=y, extra_fields=('potential_energy',))

mcmc.print_summary(exclude_deterministic=False)
pe = mcmc.get_extra_fields()['potential_energy']
print('Expected log joint density: {:.2f}'.format(np.mean(-pe)))  

sample: 100%|██████████| 1500/1500 [00:03<00:00, 483.30it/s, 7 steps of size 4.57e-01. acc. prob=0.87]



                         mean       std    median      5.0%     95.0%     n_eff     r_hat
                 mu      4.35      3.31      4.41     -0.20     11.07   1147.03      1.00
                tau      3.59      3.26      2.68      0.00      8.11    540.06      1.00
           theta[0]      6.51      5.93      5.74     -2.83     15.01    673.03      1.00
           theta[1]      4.75      4.57      4.91     -2.73     12.02    832.71      1.00
           theta[2]      3.74      5.30      4.17     -4.20     12.22    876.41      1.00
           theta[3]      4.68      4.59      4.71     -2.39     11.88    923.09      1.00
           theta[4]      3.60      4.70      3.91     -3.99     11.00   1146.94      1.00
           theta[5]      3.94      4.84      4.22     -3.19     12.03   1329.90      1.00
           theta[6]      6.20      5.13      5.70     -2.59     13.27    995.17      1.00
           theta[7]      4.87      5.20      4.90     -3.10     12.40    973.18      1.00
theta_dec

新しい別の学校に対する予測をしたい場合は次のようにかけます。

In [53]:
from numpyro.infer import Predictive

def new_school():
    mu = numpyro.sample('mu', dist.Normal(0, 5))
    tau = numpyro.sample('tau', dist.HalfCauchy(5))
    return numpyro.sample('obs', dist.Normal(mu, tau))

predictive = Predictive(new_school, mcmc.get_samples())

samples_predictive = predictive(random.PRNGKey(1))

print(np.mean(samples_predictive['obs']))  

4.4431524
